In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import skimage.io
import keras.backend as K
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization ,Activation
from tensorflow.keras.models import Model, Sequential
from keras.applications.nasnet import NASNetLarge
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
Trial = 'ResNet_new'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,                                  
    rotation_range=5,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    #zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale = 1./255)
# test_datagen  = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_dataset  = train_datagen.flow_from_directory(directory = '../input/fer2013-new/FER2013/train',
                                                   target_size = (224,224),
                                                   interpolation='bilinear',
                                                   class_mode = 'categorical',
                                                   batch_size = 64)

In [ ]:
valid_dataset = valid_datagen.flow_from_directory(directory = '../input/fer2013-new/FER2013/test',
                                                  target_size = (224,224),
                                                  interpolation='bilinear',
                                                  class_mode = 'categorical',
                                                  batch_size = 64,
                                                  shuffle=False)

In [ ]:
# test_dataset = test_datagen.flow_from_directory(directory = '../input/fer2013-new/FER2013/val',
#                                                   target_size = (224,224),
#                                                   interpolation='bilinear',
#                                                   class_mode = 'categorical',
#                                                   batch_size = 64,
#                                                   shuffle=False)

In [ ]:
model = tf.keras.applications.ResNet50(input_shape=(224,224,3),include_top=True,weights=None,classes=7)
# model.summary()

In [ ]:
lrd = ReduceLROnPlateau(monitor = 'val_loss',patience = 20,verbose = 1,factor = 0.50, min_lr = 1e-10)
mcp = ModelCheckpoint('model.h5')
es = EarlyStopping(verbose=1, patience=20)

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
# model.load_weights('../input/emotion-util/ResNet_1.h5')

In [ ]:
history=model.fit(train_dataset,validation_data=valid_dataset,epochs = 60,verbose = 1,callbacks=[lrd,mcp,es])

In [ ]:
model.save(Trial + '.h5')
history.history['lr'] = [float(i) for i in history.history['lr']]
json.dump(history.history, open(Trial+'.json','w'))

In [ ]:
train_loss = history.history['loss']
valid_loss = history.history['val_loss']
train_acc = history.history['accuracy']
valid_acc = history.history['val_accuracy']

plt.figure(figsize=(18,6))
plt.subplot(121)
plt.plot(train_loss,'-',label='train')
plt.plot(valid_loss,'-',label='val')
plt.legend(edgecolor='black',fancybox= False,fontsize=12)
plt.title('Loss');
plt.subplot(122)
plt.plot(train_acc,'-',label='train')
plt.plot(valid_acc,'-',label='val')
plt.legend(edgecolor='black',fancybox= False,fontsize=12)
plt.title('Acc.');